# **Classifier Retrieval Dataset** 

In [31]:
import sys
import os
import numpy as np

sys.path.insert(0, "..\\Scripts")

In [32]:
from ImageDatastore import ImageDatastore
from torch.utils.data import DataLoader

## **Dataset**

In [33]:
from torchvision import transforms

In [34]:
batch_size = 512

In [35]:
transforms = transforms.Compose(
    [
        transforms.Resize(
            (232, 232), interpolation=transforms.InterpolationMode.BILINEAR
        ),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [36]:
train_set = ImageDatastore('train_augmented', transform=transforms)
train_set_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

In [37]:
val_set = ImageDatastore('val_set', transform=transforms)
val_set_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)

In [38]:
val_deg = ImageDatastore('val_degradate', transform=transforms)
val_deg_loader = DataLoader(val_deg, batch_size=batch_size, shuffle=True)

## **Feature extraction**

In [39]:
from NeuralFeatureExtractor import MobileNetFeatureExtractor

In [40]:
feature_extractor = MobileNetFeatureExtractor()

In [41]:
x_train, y_train = feature_extractor.compute_features(train_set_loader)
x_train.shape, y_train.shape

95it [32:26, 20.49s/it]


((48255, 1280), (48255,))

In [42]:
# x_test, y_test = feature_extractor.compute_features(val_set_loader)
# x_test.shape, y_test.shape

In [43]:
# np.save(
#     os.path.join('..', 'Features', 'features', 'x_val_mobilenetV3.npy'),
#     x_test,
#     allow_pickle=False,
# )
# np.save(
#     os.path.join('..', 'Features', 'labels', 'y_val_mobilenetV3.npy'),
#     y_test,
#     allow_pickle=False,
# )

In [44]:
x_test = np.load(os.path.join('..', 'Features', 'features', 'x_val_mobilenetV3.npy'))
y_test = np.load(os.path.join('..', 'Features', 'labels', 'y_val_mobilenetV3.npy'))
x_test.shape, y_test.shape

((11994, 1280), (11994,))

In [45]:
# x_deg, y_deg = feature_extractor.compute_features(val_deg_loader)
# x_deg.shape, y_deg.shape

In [46]:
# np.save(
#     os.path.join('..', 'Features', 'features', 'x_deg_mobilenetV3.npy'),
#     x_deg,
#     allow_pickle=False,
# )
# np.save(
#     os.path.join('..', 'Features', 'labels', 'y_deg_mobilenetV3.npy'),
#     y_deg,
#     allow_pickle=False,
# )

In [47]:
x_deg = np.load(os.path.join('..', 'Features', 'features', 'x_deg_mobilenetV3.npy'))
y_deg = np.load(os.path.join('..', 'Features', 'labels', 'y_deg_mobilenetV3.npy'))
x_deg.shape, y_deg.shape

((11994, 1280), (11994,))

## **Models**

### **k-NN Classifier**

In [48]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, top_k_accuracy_score

In [49]:
knn = KNeighborsClassifier(n_neighbors=51, n_jobs=-1, weights='distance', metric='cosine')

In [50]:
knn.fit(x_train, y_train)   

KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=51,
                     weights='distance')

In [51]:
knn_pred = knn.predict(x_test)
print(f"Val: {accuracy_score(y_test, knn_pred)}")

Val: 0.28597632149408037


In [52]:
knn_score = knn.predict_proba(x_test)
print(f"Val top 5: {top_k_accuracy_score(y_test, knn_score, k=5)}")

Val top 5: 0.5253460063365016


In [53]:
knn_pred_deg = knn.predict(x_deg)
print(f"Deg: {accuracy_score(y_deg, knn_pred_deg)}")

Deg: 0.2183591795897949


In [54]:
knn_score_deg = knn.predict_proba(x_deg)
print(f"Deg top 5: {top_k_accuracy_score(y_deg, knn_score_deg, k=5)}")

Deg top 5: 0.41412372853093216


### **Tree Classifier**

In [55]:
from sklearn.ensemble import RandomForestClassifier

tree = RandomForestClassifier(criterion='gini', n_estimators=100, n_jobs=-1)

In [56]:
tree.fit(x_train, y_train)

RandomForestClassifier(n_jobs=-1)

In [57]:
tree_pred = tree.predict(x_test)
print(accuracy_score(y_test, tree_pred))

0.21310655327663833


In [58]:
tree_score = tree.predict_proba(x_test)
print(top_k_accuracy_score(y_test, tree_score, k=5))

0.40445222611305653


In [59]:
tree_pred_deg = tree.predict(x_deg)
print(accuracy_score(y_deg, tree_pred_deg))

0.15232616308154076


In [60]:
tree_score_deg = tree.predict_proba(x_deg)
print(top_k_accuracy_score(y_deg, tree_score_deg, k=5))

0.3002334500583625


### **Ensemble Classifier**